In [1]:
# Copyright (C) 2018-2021 Intel Corporation
# SPDX-License-Identifier:  BSD-3-Clause

_Motivation: In this tutorial, we will build a Lava Process for an MNIST 
classifier, using the Lava Processes for LIF neurons and Dense connectivity. 
We will see how a single MNIST Lava Process runs on Tensorflow software 
backend. We will use the `ExecVar` mechanism to read/write neuron states 
directly._

# An MNIST Classifier as a Hierarchical Process in Lava, using LIF and Dense Processes

### This tutorial assumes that you:
- have the Lava framework installed 
- are familiar with the *Process* concept in Lava (please refer to other Lava tutorials)
- know the ANN to SNN conversion philosophy using rate coding or any other type of encoding

### This tutorial shows 
- how a Lava Process can be run on Tensorflow backend
- how to interact with a Lava process via `ExecVar`s, to read and write 
neuronal states

### This tutorial does not
- show how to train an MNIST classifier
- show how to convert an ANN classifier to SNN classifier

### Our MNIST Classifier
For the purposes of this tutorial we have chosen a classifier, which does not use any convolutional layers. It has a simple feed-forward architecture with ReLU activations and all-to-all dense connectivity in case of an ANN, and LIF neurons with all-to-all dense connectivity in case of an SNN, as shown below.

![image](mnist_arch.png)

Note that connectivity and activation are both specified as parameters of a single layer object in an ANN implemented in Keras, whereas in a Lava Process, neuron and connection Processes are two different objects. This is indicated with a colour gradient in the ANN architecture as opposed to two distinct colours in the SNN architecture.

### A note on network training and obtaining parameters for the SNN classifier

Instead of directly training the SNN classifier, we have trained the equivalent ANN in Keras using standard backpropagation. Then we have used SNNToolBox to convert the ANN to an SNN model. The model parameters of the SNN model are saved to disk.

When we wish to run inference using the Lava Process, we simply load the pre-trained model parameters from the disk and run the Process. The model parameters not only include weights and biases, but also the LIF neuron parameters like threshold voltage. This flow is illustrated in the schematic below, along with the corresponding accuracies on a validation set:

![image](ann_snn_conversion.png)

In [2]:
# Assumes: lava root is in $PYTHONPATH
import os
import numpy as np
from lava.core.generic.process import Process
from lava.processes.generic.dense import Dense
from lava.processes.generic.lif import LIF 
from lava.core.generic.enums import Backend

# DatasetUtil class downloads MNIST data using Keras
# and stores it in lava-mnist/datasets/MNIST directory
from datasets.dataset_utils import DatasetUtil

# DatasetBuffer class holds MNIST data
from nets.mnist_execvar_io import DatasetBuffer

2021-07-09 15:17:43.886197: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /nfs/slurm/intel-archi/lib
2021-07-09 15:17:43.886261: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


#### MNIST dataset
We use a Keras convenience function to get the MNIST data and store it in
`lava-mnist/datasets/MNIST` directory. Run the following cell if the dataset
is locally unavailable in this directory.

In [3]:
db = DatasetUtil()
db.save_npz()

11493376/11490434 [==============================] - 1s 0us/step
Creating /home/dflorey/os3_lava/lava-mnist/datasets/MNIST


### Building processes

#### MNIST Process:

Every hierarchical Process inherits from `Process` class. Every process needs to define its own `_build()` method, which contains the architectural specification of the proess.

##### Note: 
As mentioned above, the `classify()` method exists as a separate method,
because this tutorial deals with direct input injection and output readout
using the `ExecVar` mechanism. This method performs the job of input
injection by explicit register writes (the line that says `self.input_lif
.bias_mant.value = new_biases`) as well as register readouts (the line that
says `voltages = self.output_lif.v.value`).

In [4]:
class SimpleMnist(Process):
    """A simple MNIST digit classifying process, comprised by only LIF and
    Dense processes. The architecture is: Input (784,) -> Dense (64,
    ) -> Dense(64,) -> Dense(10,)"""
    def _build(self, **kwargs):
        dataset_path = kwargs.pop('dataset_path', '../datasets/MNIST')
        trained_wgts = kwargs.pop('trained_weights_path',
                                  './mnist_dense_only_snn_wblist.npy')
        steps_per_image = kwargs.pop('num_steps_per_image', 512)

        ###
        # Code to read and load weights and biases; LIF parameters
        # vth, bias_exp, dv, du are directly read from the SNN
        # model generated by SNNToolBox.
        ###
        real_path_trained_wgts = os.path.realpath(trained_wgts)
        wb_list = np.load(real_path_trained_wgts, allow_pickle=True)
        w_dense1 = wb_list[0].transpose().astype(np.int32)
        b_lif1 = wb_list[1].astype(np.int32)
        w_dense2 = wb_list[2].transpose().astype(np.int32)
        b_lif2 = wb_list[3].astype(np.int32)
        w_dense3 = wb_list[4].transpose().astype(np.int32)
        b_lif3 = wb_list[5].astype(np.int32)

        ###
        # Network definition
        ###
        self.data_buff = DatasetBuffer(dataset_path=dataset_path)
        self.input_lif = LIF(size=784, bias_exp=6, vth=1528,
                             dv=0, du=4095)
        
        self.dense1 = Dense(wgts=w_dense1)(s_in=self.input_lif.s_out)
        self.lif1 = LIF(size=self.dense1.size[0], bias_mant=b_lif1,
                        bias_exp=6, vth=380, dv=0, du=4095)(a_in=self.dense1.a_out)
        self.dense2 = Dense(wgts=w_dense2)(s_in=self.lif1.s_out)
        self.lif2 = LIF(size=self.dense2.size[0], bias_mant=b_lif2,
                        bias_exp=6, vth=354, dv=0, du=4095)(a_in=self.dense2.a_out)
        self.dense3 = Dense(wgts=w_dense3)(s_in=self.lif2.s_out)
        
        self.output_lif = LIF(size=self.dense3.size[0], bias_mant=b_lif3,
                              bias_exp=6, vth=131071, dv=0, du=4095)(a_in=self.dense3.a_out)
        self.predictions = []
        self.ground_truths = []

    def reset_process(self):
        """Reset the neuronal states bias, current, and voltage for 
        all LIF processes. This is done after every input, to flush 
        the old states."""
        self.input_lif.bias_mant.value = np.zeros((self.input_lif.size,),
                                                  dtype=np.int32)
        self.input_lif.u.value = np.zeros((self.input_lif.size,),
                                          dtype=np.int32)
        self.input_lif.v.value = np.zeros((self.input_lif.size,),
                                          dtype=np.int32)
        self.lif1.u.value = np.zeros((self.lif1.size,), dtype=np.int32)
        self.lif1.v.value = np.zeros((self.lif1.size,), dtype=np.int32)
        self.lif2.u.value = np.zeros((self.lif2.size,), dtype=np.int32)
        self.lif2.v.value = np.zeros((self.lif2.size,), dtype=np.int32)
        self.output_lif.u.value = np.zeros((self.output_lif.size,), dtype=np.int32)
        self.output_lif.v.value = np.zeros((self.output_lif.size,), dtype=np.int32)

    @staticmethod
    def compute_new_biases(raw_x_data, img_id, size):
        """Convert raw input image data to integer bias values. Each 
        input value is scaled to [0, 255]."""
        new_img_bias = np.int32((raw_x_data[img_id, :]) * 255)
        new_img_bias = new_img_bias.reshape((size,))
        return new_img_bias

    def classify(self, backend=Backend.TF, n_img=10, n_st_img=512):
        """Loop over all n_img number of input images, with n_st_img 
        steps per input."""
        
        print( '----------------------------------------------------', flush=True)
        print(f'Running MNIST classifier with {backend.name} backend', flush=True)
        print( '----------------------------------------------------', flush=True)

        # Running for 0 time-steps just compiles the Process
        self.run(num_steps=0, backend=backend)
        
        for img_id in range(n_img):
            print(f'Classifying img id: {img_id}... ', end='', flush=True)
            
            # Compute biases for InputLIF neurons
            new_biases = self.compute_new_biases(
                raw_x_data=self.data_buff.x_test, img_id=img_id,
                size=self.data_buff.out_size)
            
            # Write biases to InputLIF
            self.input_lif.bias_mant.value = new_biases
            
            # Run the classifier for this input
            self.run(num_steps=n_st_img, backend=backend)
            
            # Read out the voltage state of OutputLIF
            voltages = self.output_lif.v.value
            self.predictions.append(np.argmax(voltages))
            self.ground_truths.append(np.argmax(self.data_buff.y_test[img_id]))
            self.reset_process()
            print(f' Done.\tPredicted Label: {np.argmax(voltages)}\tGround '
                  f'Truth: {np.argmax(self.data_buff.y_test[img_id])}')

        print(self.predictions, self.ground_truths)
        accuracy = np.sum(np.array(self.predictions) == np.array(
            self.ground_truths)) / len(self.ground_truths) * 100
        print(f'{backend.name} execution was {accuracy} % accurate.')

### Run classification on Loihi and Tensorflow

In [5]:
data_path = '../datasets/MNIST'
wgts_path = '../trained_models/mnist_dense_only_snn_wblist.npy'
num_steps_per_image = 512
num_images = 10

# Instantiate the classifier
mnist_classifier = SimpleMnist(dataset_path=data_path,
                               trained_weights_path=wgts_path,
                               num_steps_per_image=num_steps_per_image)

# Run on TF. Note how the backend is specified.
mnist_classifier.classify(backend=Backend.TF, n_img=num_images,
                          n_st_img=num_steps_per_image)

INFO:  If weight tensor is sparse, use Sparse() process and pass the weight tensor as a tensorflow.SparseTensor.
INFO:  If weight tensor is sparse, use Sparse() process and pass the weight tensor as a tensorflow.SparseTensor.
INFO:  If weight tensor is sparse, use Sparse() process and pass the weight tensor as a tensorflow.SparseTensor.
----------------------------------------------------
Running MNIST classifier with TF backend
----------------------------------------------------
Classifying img id: 0... WARNING:tensorflow:From /home/dflorey/os3_lava/lava-tf-compiler/lava/core/tf/executor.py:270: experimental_run_functions_eagerly (from tensorflow.python.eager.def_function) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


2021-07-09 15:17:54.769205: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /nfs/slurm/intel-archi/lib
2021-07-09 15:17:54.769241: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-07-09 15:17:54.769265: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ncl-dev-01): /proc/driver/nvidia/version does not exist
2021-07-09 15:17:54.769610: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


 Done.	Predicted Label: 7	Ground Truth: 7
Classifying img id: 1...  Done.	Predicted Label: 2	Ground Truth: 2
Classifying img id: 2...  Done.	Predicted Label: 1	Ground Truth: 1
Classifying img id: 3...  Done.	Predicted Label: 0	Ground Truth: 0
Classifying img id: 4...  Done.	Predicted Label: 4	Ground Truth: 4
Classifying img id: 5...  Done.	Predicted Label: 1	Ground Truth: 1
Classifying img id: 6...  Done.	Predicted Label: 8	Ground Truth: 4
Classifying img id: 7...  Done.	Predicted Label: 9	Ground Truth: 9
Classifying img id: 8...  Done.	Predicted Label: 8	Ground Truth: 5
Classifying img id: 9...  Done.	Predicted Label: 9	Ground Truth: 9
[7, 2, 1, 0, 4, 1, 8, 9, 8, 9] [7, 2, 1, 0, 4, 1, 4, 9, 5, 9]
TF execution was 80.0 % accurate.
